<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt

# Finite Volume Methods

## General Formulation of a Conservation Law

Define the value
$$
    Q^n_i \approx \frac{1}{\Delta x} \int^{x_{i+1/2}}_{x_{i-1/2}} q(x, t_n) dx \equiv \frac{1}{\Delta x} \int_{\mathcal{C}_i} q(x, t_n) dx
$$
where the $i$th **grid cell** is denoted by
$$
    \mathcal{C}_i \equiv (x_{i-1/2}, x_{i+1/2})
$$
and $\Delta x = x_{i+1/2} - x_{i-1/2}$.  

If $q(x,t)$ is smooth then the average value 
$$
    Q^n_i = q(x_i, t_n) + \mathcal{O}(\Delta x^2)
$$

With this definition we now return to our version of the conservation  law written over the cell $\mathcal{C}_i$:
$$
    \frac{\text{d}}{\text{d} t} \int_{\mathcal{C}_i} q(x,t) dx = f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)).
$$
Now take the following steps:
$$\begin{aligned}
    \frac{\text{d}}{\text{d} t} \int_{\mathcal{C}_i} q(x,t) dx &= f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)) & & \text{Original}\\
    \int_{\mathcal{C}_i} q(x,t_{n+1}) dx - \int_{\mathcal{C}_i} q(x,t_{n}) dx &= \int^{t_{n+1}}_{t_n} \left[ f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)) \right ] dt & & \text{Integrate over } [t_n, t_{n+1}] \\
    \frac{1}{\Delta x} \int_{\mathcal{C}_i} q(x,t_{n+1}) dx &= \frac{1}{\Delta x} \int_{\mathcal{C}_i} q(x,t_{n}) dx + \frac{1}{\Delta x} \int^{t_{n+1}}_{t_n} \left[ f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)) \right ] dt & & \text{Rearrange and divide by } \Delta x \\
    Q^{n+1}_i &= Q^n_i + \frac{1}{\Delta x} \int^{t_{n+1}}_{t_n} \left[ f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)) \right ] dt & & \text{Approximate with averages } Q^{n+1,n}_i \\
    Q^{n+1}_i &= Q^n_i - \frac{\Delta t}{\Delta x} (F^n_{i+1/2} - F^n_{i-1/2}) & & \text{Replace with }F_{i\pm1/2} \\
\end{aligned}$$
where
$$
    F^n_{i\pm 1/2} \approx \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} f(q(x_{i\pm 1/2}, t)) dt.
$$

This approximate flux should have a formula akin to
$$
    F^n_{i-1/2} = \mathcal{F}(Q^n_{i-1}, Q^n_i)
$$
where here we are thinking of $\mathcal{F}$ as a numerical flux function.  The question then turns to how to prescribe a **numerical flux function**.  Note that this implies that the update to $Q^{n+1}_i$ depends on three previous values, $Q^n_{i-1}, Q^n_{i}, Q^n_{i+1}$.

We can also now discuss how conservation should be described numerically.  If
$$
    \Delta x \sum^J_{i=I} Q^{n+1}_i = \Delta x \sum^J_{i=I} Q^n_i - \frac{\Delta t}{\Delta x} (F^n_{J+1/2} - F^n_{I-1/2})
$$
and the sum of the fluxes cancels then **global conservation** is maintained up to the boundaries of the domain.

We can also rewrite the numerical methods as
$$
    \frac{Q^{n+1}_i - Q^n_i}{\Delta t} + \frac{F^n_{i+1/2} - F^n_{i-1/2}}{\Delta x} = 0
$$
which implies that we have formulated a first order method in terms of finite difference methods.  We will find that this is not entirely the case depending on what is prescribed for $F_{i\pm 1/2}$.

## Numerical Flux for the Diffusion Equation

An interesting exercise at this point is to consider a more general method to the above approach.  Consider again the parabolic heat equation.  We can define a flux function that contains a derivative of $q$ such that
$$
    f(q_x, x) = -\beta(x) q_x.
$$
Cleary this gives us the heat equation if we substitute this flux function into our general equations
$$
    q_t + f(q_x, x)_x = q_t - (\beta(x) q_x)_x = 0.
$$

One way to prescribe a numerical flux would be the following:
$$
    \mathcal{F}(Q_{i-1}, Q_i) = -\beta_{i-1/2} \left ( \frac{Q_i - Q_{i-1}}{\Delta x} \right ).
$$
If we use this in our previous definition we then have
$$
    Q^{n+1}_i = Q^n_i + \frac{\Delta t}{\Delta x^2} \left[ \beta_{i+1/2}(Q^n_{i+1} - Q^n_i) - \beta_{i-1/2} (Q^n_i - Q^n_{i-1}) \right]
$$
noting that if $\beta$ is constant we have
$$
    Q^{n+1}_i = Q^n_i + \beta \frac{\Delta t }{\Delta x^2} \left[ Q^n_{i+1} - 2 Q^n_i + Q^n_{i-1} \right]
$$

We however know that the previous scheme is not ideal.  Instead if we use the well-known Crank-Nicolson scheme we have
$$
    Q^{n+1}_i = Q^n_i + \frac{\Delta t}{2 \Delta x^2} \left[ \beta_{i+1/2}(Q^n_{i+1} - Q^n_i) - \beta_{i-1/2} (Q^n_i - Q^n_{i-1}) + \beta_{i+1/2}(Q^{n+1}_{i+1} - Q^{n+1}_i) - \beta_{i-1/2} (Q^{n+1}_i - Q^{n+1}_{i-1})\right ]
$$
with the flux
$$
    F^n_{i-1/2} = -\frac{1}{\Delta x} \left[ \beta_{i-1/2} (Q^n_i - Q^n_{i-1}) + \beta_{i-1/2} (Q^{n+1}_i - Q^{n+1}_{i-1}) \right ]
$$

## Convergence

For any numerical method we desire that as $\Delta x, \Delta t \rightarrow 0$ that the numerical solution converges to the true solution.  This generally requires the following conditions:

1. The method must be consistent:  the approximation is valid locally.
1. The method must be stable:  small errors do not accumalate too fast.

### Consistency

In the case of numerical fluxes we want to require that the numerical flux reduces to the true flux in some sense.  One way to require this is to ensure for regions where $q$ is constant that the numerical flux agrees with the flux function:
$$
    \mathcal{F}(q, q) = f(q).
$$
A more formal definition is to ensure that there is some sort of Lipschitz continuity of the form
$$
    |\mathcal{F}(Q_{i-1}, Q_i) - f(q)| \leq L \text{max}(|Q_i - q|, |Q_{i-1} - q|).
$$

### Stability

Stability can take on many different forms, many of which will be discussed later.  Here we will consider the necessary condition of CFL stability.  This is usually expressed as

$$
    \nu \equiv \left |\frac{u \Delta t}{\Delta x} \right | \leq 1.
$$

## Numerical Fluxes

We now will consider a number of different flux defintions and consider their viability.

### Example: Unstable Flux

Consider the flux
$$
    F^n_{i-1/2} = \mathcal{F}(Q^n_{i-1}, Q^n_i) = \frac{1}{2} [f(Q^n_{i-1}) + f(Q^n_i)]
$$
leading to the method
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{2 \Delta x} [f(Q^n_{i+1}) + f(Q^n_{i-1})].
$$

Unfortunately this method is unstable!

### Example: Lax-Friedrichs Method

The classical Lax-Friedrichs method has the flux function
$$
    \mathcal{F}(Q^n_{i-1}, Q^n_i) = \frac{1}{2} [f(Q^n_{i-1}) + f(Q^n_i)] - \frac{\Delta x}{2 \Delta t} (Q^n_i - Q^n_{i-1})
$$
and the full method
$$
    Q^{n+1}_i = \frac{1}{2} (Q^n_{i-1} + Q^n_{i+1}) - \frac{\Delta t}{2 \Delta x} [ f(Q^n_{i+1}) - f(Q^n_{i-1})]
$$
leading to a first order accurate method.

### Example: Upwind Methods

We know from finite difference methods that upwind methods have a significant advantage over more general methods by simply looking at where the flow is coming from thereby satisfying the CFL condition.  In terms of fluxes the upwind method for advection is
$$
    F^n_{i-1/2} = u Q^n_{i-1}
$$
if $u \ge 0$.  Using this flux function gives us the standard upwind method
$$
    Q^{n+1}_i = Q^n_i - \frac{u \Delta t}{\Delta x} (Q^n_i - Q^n_{i-1}).
$$

Note that the last equation has a difference of the values of $Q$ meaning that we can write this in terms of our waves.  In this context the numerical method can be written as
$$
    Q^{n+1}_i = Q^n_i - \frac{u \Delta t}{\Delta x} \mathcal{W}_{i-1/2}.
$$

We can also write the method in terms of $u \leq 0$ as
$$
    Q^{n+1}_i = Q^n_i - \frac{u \Delta t}{\Delta x} \mathcal{W}_{i+1/2}.
$$

We can generalize this so that we can define a flux function
$$
    F^n_{i-1/2} = u^- Q^n_i + u^+ Q^n_{i-1}
$$
where
$$
    u^+ = \text{max}(u, 0) \text{   and    } u^- = \text{min}(u, 0)
$$
allowing us to write a more general method as
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} (u^+ \mathcal{W}_{i-1/2} + u^- \mathcal{W}_{i+1/2})
$$

## Godunov's Method

One of the most well-known approaches for constructing a solution to a Riemann problem and a cell update is **Godunov's method**.  For linear advection upwind is a special case of Godunov's method.

### REA Algorithm

One way to view Godunov's method is in the algorithmic steps

1. Reconstruct
1. Evolve
1. Average

We will go through each step in detail next.

#### 1. Reconstruct

Reconstruct a piecewise polynomial function $\widetilde{q}^n(x, t_n)$ defined $\forall x \in \mathcal{C}_i$ from the cell averages $Q^n_i$.
$$
    \frac{1}{\Delta x} \int_{\mathcal{i}} \widetilde{q}^n(x, t_n) dx = Q^n_i
$$
In the simplest case we use a piecewise constant and we therefore have
$$
    \widetilde{q}^n(x, t_n) = Q^n_i \quad \forall x \in \mathcal{C}_i.
$$

#### 2. Evolve

Evolve the hyperbolic equation exactly (or approximately) with this initial data to obtain $\widetilde{q}^n(x, t_{n+1})$ a time $\Delta t$ later.

#### 3. Average

Average this new function $\widetilde{q}^n(x, t_{n+1})$ over the grid cell $\mathcal{C}_i$ to obtain the new cell-average:
$$
    Q^{n+1}_i = \frac{1}{\Delta x} \int_{\mathcal{C}_i} \widetilde{q}^n(x, t_{n+1}) dx.
$$

One of the key points to note in Godunov's method is that in step 2, evolve, we are solving the hyperbolic equation nominally exactly.  In the simple case of the piecewise constants we have set of Riemann problems to solve, hence their importance in our ongoing discussion.

### Time Step Constraints

What kind of time constraints arise naturally from Godunov's method?  Drawing some diagrams we may think that we need to take timesteps limited by
$$
    c \Delta t \leq \frac{1}{2} \Delta x.
$$

### Numerical Flux for Godunov's Method

So far we have been able to write an effective numerical flux function for each approach we have discussed.  The same can be said for Godunov's method.  The basic formula for a flux should look like
$$
    F^n_{i-1/2} \approx \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} f(q(x_{i-1}, t)) dt.
$$

The most direct thing we could do is to replace the value of $q(x_{i-1}, t)$ with our reconstructed function such that
$$\begin{aligned}
    F^n_{i-1/2} \approx& \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} f(q(x_{i-1}, t)) dt \\
    = & \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} f(\widetilde{q}(x_{i-1}, t)) dt
\end{aligned}$$
What does this integrate to now?

Recall that for a general Riemann solution we have a set of states in between each wave.  Inside this wedge the solution $\widetilde{q}(x_{i-1}, t)$ is constant.  If we define the value along the grid cell edge at $x_{i-1/2}$ as $\widehat{q}(Q^n_{i-1}, Q^n_i)$ we can then find an expression for the flux as

$$\begin{aligned}
    F^n_{i-1/2} \approx& \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} f(\widetilde{q}(x_{i-1}, t)) dt \\
    & = f(\widehat{q}(Q^n_{i-1}, Q^n_i)).
\end{aligned}$$

### Wave-Propagation Form of Godunov's Method

Writing Godunov's method in terms of wave-propagation has some advantages later on when we consider non-conservative hyperbolic PDEs and non-linear cases.  To that end consider the definition of our waves from before as
$$
    Q_i - Q_{i-1} = \sum^m_{p=1} \alpha^p_{i-1/2} \alpha^p_{i-1/2} r^p \equiv \sum^m_{p=1} \mathcal{W}^p_{i-1/2}.
$$

Recall that we can find any middle state starting from the left or right states, $q_\ell$ or $q_r$ respectively, in the Riemann fan.

For example, if we are crossing $\mathcal{W}^2_{i-1/2} - \alpha^2_{i-1/2} r^2$ say, we know that the wave propagates at speed $\lambda^2$ and crosses $\lambda^2 \Delta t$ over the time step.  Averaging this impact into the grid cell then would lead to
$$
        -\frac{\lambda^2 \Delta t}{\Delta x} \mathcal{W}^2_{i-1/2}.
$$

Taking this to the logical conclusion we then can write the solution as
$$\begin{aligned}
    Q^{n+1}_i =& Q^n_i - \frac{\lambda^2 \Delta t}{\Delta x} \mathcal{W}^2_{i-1/2} - \frac{\lambda^3 \Delta t}{\Delta x} \mathcal{W}^3_{i-1/2} - \frac{\lambda^1 \Delta t}{\Delta x} \mathcal{W}^2_{i+1/2} \\
    &=Q^n_i - \frac{\Delta t}{\Delta x} (\lambda^2 \mathcal{W}^2_{i-1/2} + \lambda^3 \mathcal{W}^3_{i-1/2} + \lambda^1 \mathcal{W}^3_{i+1/2})
\end{aligned}$$

Define $\lambda^+ \equiv \max(\lambda, 0)$ and $\lambda^- \equiv \min(\lambda, 0)$ we can write this in general as
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} \left[ \sum^m_{p=1} (\lambda^p)^+ \mathcal{W}^p_{i-1/2} + \sum^m_{p=1} (\lambda^p)^- \mathcal{W}^p_{i+1/2} \right ].
$$
At this point we can identify the numerical fluxes that are now defined as
$$
    F^n_{i-1/2} = \sum^m_{p=1} (\lambda^p)^+ \mathcal{W}^p_{i-1/2}.
$$

We will also introduce the notation at this point of the **fluctuations**
$$
    \mathcal{A}^- \Delta Q_{i-1/2} = \sum^m_{p=1} (\lambda^p)^- \mathcal{W}^p_{i-1/2} \\
    \mathcal{A}^+ \Delta Q_{i-1/2} = \sum^m_{p=1} (\lambda^p)^+ \mathcal{W}^p_{i-1/2}
$$
so that the previous update formula can be written as
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} \left[ \mathcal{A}^+ \Delta Q_{i-1/2} + \mathcal{A}^- \Delta Q_{i+1/2} \right ].
$$
These fluctuations represent the total impact of the left and right going waves.  

This additional notation can be motivated via the linear case.  If we define the matrices
$$
    \Lambda^+ = \text{diag}[(\lambda^p)^+] \quad \Lambda^- = \text{diag}[(\lambda^p)^-]
$$
we can then also write split matrices for $A$ as
$$
    A^+ = R \Lambda^+ R^{-1} \quad A^- = R \Lambda^- R^{-1}
$$
noting that
$$
    A = R \Lambda R^{-1} = R (\Lambda^+ + \Lambda^-) R^{-1} = A^+ + A^-.
$$

Turning back to our notation, we can now write the multiplication
$$\begin{aligned}
    A^+ \Delta Q_{i-1/2} &= R \Lambda^+ R^{-1} (Q_i - Q_{i-1}) \\
    =& R \Lambda^+ \alpha_{i-1/2} \\
    =& \sum^m_{p=1} (\lambda^p) \alpha^p_{i-1/2} r^p \\
    =& \mathcal{A}^+ \Delta Q_{i-1/2}.
\end{aligned}$$

### Roe's Method

One additional and helpful way to look at the definition of the fluxes is to rewrite
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} \left[ \sum^m_{p=1} (\lambda^p)^+ \mathcal{W}^p_{i-1/2} + \sum^m_{p=1} (\lambda^p)^- \mathcal{W}^p_{i+1/2} \right ]
$$
or the flux version
$$
    F^n_{i-1/2} = \sum^m_{p=1} (\lambda^p)^+ \mathcal{W}^p_{i-1/2}
$$
as
$$
    F^n_{i-1/2} = \frac{1}{2} \left [ (AQ_{i-1} + AQ_i) - \sum^m_{p=1} [(\lambda^p)^+ - (\lambda^p)^- ] \mathcal{W}^p_{i-1/2} \right].
$$

Define
$$
    |A| = R |\Lambda| R^{-1|} \quad |\Lambda| = \text{diag}(|\lambda^p|)
$$
we then have
$$\begin{aligned}
    F^n_{i-1/2} &= \frac{1}{2} \left [ (AQ_{i-1} + AQ_i) - \sum^m_{p=1} [(\lambda^p)^+ - (\lambda^p)^- ] \mathcal{W}^p_{i-1/2} \right] \\
    &= \frac{1}{2}(A Q_{i-1} + A Q_i) - \frac{1}{2} |A| (Q_i - Q_{i-1}) \\
    &= \frac{1}{2} [f(Q_{i-1} + f(Q_i)] - \frac{1}{2} |A| (Q_i - Q_{i-1})
\end{aligned}$$
hence an average of of the fluxes we know to be unstable *plus* a correction term.